In [2]:
# import lib
import numpy as np 
import tensorflow as tf
import matplotlib.pyplot as plt

np.random.seed(78)
tf.set_random_seed(678)

In [8]:
# declare
num_epoch    = 10
ground_truth = np.random.uniform(0,1,10)
expected     = np.zeros(10)
print(ground_truth)
print(expected)

[0.91097965 0.3443528  0.14967316 0.55192749 0.48696221 0.29101638
 0.01202189 0.24147763 0.32063698 0.24400811]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [29]:
# train 
for iter in range(num_epoch):
    current_action = np.random.randint(10)
    print(current_action)    
    print(ground_truth[current_action])
    print(np.random.binomial(1, ground_truth[current_action]))
    print('------')


0
0.9109796506321873
0
------
7
0.2414776262567656
0
------
3
0.5519274902548998
0
------
9
0.24400811383276544
0
------
7
0.2414776262567656
0
------
5
0.291016382189513
0
------
1
0.3443528034183553
0
------
8
0.3206369844038238
0
------
7
0.2414776262567656
1
------
0
0.9109796506321873
1
------


# Reference 
1. Solving the Multi-Armed Bandit Problem – Towards Data Science. (2017). Towards Data Science. Retrieved 24 December 2018, from https://towardsdatascience.com/solving-the-multi-armed-bandit-problem-b72de40db97c
2. 